# A new Pareto set generation method
This notebook contains python implementation of the examples mentioned in the article `A new Pareto set generating method for multi-criteria optimization problems` by Debdas Ghosh and Debjani Chakraborty (DOI: https://doi.org/10.1016/j.orl.2014.08.011).

This implementation uses [KNITRO](https://www.artelys.com/solvers/knitro/) solver on [NEOS server](https://neos-server.org/neos/solvers/index.html) and requires no additional setup other than an active internet connection.

## Installing dependencies
You can skip this section if you've installed packages `pyomo` and `matplotlib`. Refer [this link](http://www.pyomo.org/installation) for `pyomo` third party dependencies.

In [ ]:
import sys
!{sys.executable} -m pip install pyomo matplotlib

## Simulation

In [ ]:
from __future__ import division
from pyomo.environ import *
from math import pi as PI
from math import cos as COS
from math import sin as SIN

In [ ]:
m = 10  # number of grid points
K = 2   # problem dimensionality

In [ ]:
def solve(beta):
    model = ConcreteModel()

    # generic variables
    model.z = Var(domain=Reals, initialize=0.05) # init with z=0 fails on 1st step. refer article
    model.OBJ = Objective(expr = model.z, sense=minimize)

    # problem specific variables
    model.x1 = Var(domain=Reals, bounds=(0,PI))
    model.x2 = Var(domain=Reals, bounds=(0,PI))
    f = (model.x1, model.x2)     # multicriterion objective function

    # add generic beta constraints
    model.DIM = RangeSet(0,K-1)
    def beta_constraint(model, i):
        return model.z*beta[i] >= f[i]
    model.beta_const = Constraint(model.DIM,rule=beta_constraint)

    # add problem specific feasibility constraints
    model.C1 = Constraint(expr = (model.x1-0.5)**2 + (model.x2-0.5)**2 <= 0.5)
    model.C2 = Constraint(expr = model.x1**2 + model.x2**2 -1 -0.1*cos(16*atan(model.x1/model.x2))>= 0)
    
    # model.pprint()
    
    # solve on NEOS solver 
    solver_manager = SolverManagerFactory('neos')
    results = solver_manager.solve(model, opt='knitro')
    
#     print(results)
    
    if results.solver.status == SolverStatus.ok and results.solver.termination_condition == TerminationCondition.optimal:
        return value(model.OBJ)
    elif results.solver.termination_condition == TerminationCondition.infeasible:
        return 0
    else:
        return 0

In [ ]:
# do the simulation
points = []

for i in range(m+1): # 0,m inclusive
    theta1 = (i*PI)/(4*m)
    beta = (COS(theta1), SIN(theta1))
    tmp = solve(beta)
    print("iter %d: z=%f" % (i,tmp))
    points.append((tmp,beta))

## Analysing result

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as pat

fig, ax = plt.subplots()
ax.set_aspect('equal')

# drawing the circle
circle = pat.Wedge((0.5,0.5), 0.707,-45,135,color='grey')
circle.set_clip_on(False)
circle.set_alpha(0.3)
ax.add_artist(circle)

for z, beta in points:
    if z > 0: # coz the function is symmetric
        plt.scatter(z*beta[0], z*beta[1], color='black')
        plt.scatter(z*beta[1], z*beta[0], color='black')
plt.show()